<a href="https://colab.research.google.com/github/irfan117/tugaspyhton/blob/main/CompisMETlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import file

In [1]:
from google.colab import files

uploaded = files.upload()


Saving ras.zip to ras.zip


In [2]:
from google.colab import files
import zipfile
import os

# Path tempat untuk mengekstrak dataset
extract_path = '/content/dataset'  # Ubah sesuai kebutuhan

# Membuat folder ekstraksi jika belum ada
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Path file zip yang diupload (ras.zip)
zip_path = '/content/ras.zip'

# Mengekstraksi file zip ke dalam folder dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Tampilkan daftar file setelah diekstraksi
extracted_files = os.listdir(extract_path)
print("Dataset telah diekstraksi ke:", extract_path)
print("Daftar file setelah ekstraksi:", extracted_files)



Dataset telah diekstraksi ke: /content/dataset
Daftar file setelah ekstraksi: ['ras']


In [6]:
# Tentukan kategori label
categories = ['African', 'American', 'Asia tenggara', 'Asia Timur', 'Eropa']


In [7]:
import os

def label_dataset(dataset_path, categories):
    labeled_data = []

    # Iterasi melalui setiap kategori label
    for i, category in enumerate(categories):
        category_path = os.path.join(dataset_path, category)

        # Iterasi melalui setiap file gambar dalam folder kategori
        for filename in os.listdir(category_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(category_path, filename)
                labeled_data.append((image_path, i))  # Tambahkan tuple (path, label)

    return labeled_data


In [8]:
# Path ke folder dataset yang telah diekstraksi
dataset_path = '/content/dataset/ras'

# Melabeli dataset
labeled_dataset = label_dataset(dataset_path, categories)


In [9]:
from sklearn.model_selection import train_test_split

# Pisahkan path gambar dan label dari labeled_dataset
X = [data[0] for data in labeled_dataset]
y = [data[1] for data in labeled_dataset]

# Bagi dataset menjadi data pelatihan dan data validasi
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
import numpy as np
from PIL import Image

def preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Normalisasi intensitas piksel
    return image

# Preprocessing data untuk data pelatihan dan data validasi
X_train_processed = np.array([preprocess_image(img_path) for img_path in X_train])
X_valid_processed = np.array([preprocess_image(img_path) for img_path in X_valid])


In [11]:
import numpy as np

# Konversi list y_train dan y_valid menjadi numpy array
y_train = np.array(y_train)
y_valid = np.array(y_valid)


In [32]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definisikan model CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(categories), activation='softmax')  # Jumlah output sesuai dengan jumlah kategori label
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Latih model menggunakan data pelatihan
model.fit(X_train_processed, y_train, epochs=10, validation_data=(X_valid_processed, y_valid))


Epoch 1/10
5/5 [==============================] - 20s 4s/step - loss: 2.6259 - accuracy: 0.1875 - val_loss: 1.6019 - val_accuracy: 0.2222
Epoch 2/10
5/5 [==============================] - 17s 3s/step - loss: 1.5434 - accuracy: 0.2708 - val_loss: 1.9713 - val_accuracy: 0.3056
Epoch 3/10
5/5 [==============================] - 16s 3s/step - loss: 1.5176 - accuracy: 0.5278 - val_loss: 1.5318 - val_accuracy: 0.3056
Epoch 4/10
5/5 [==============================] - 17s 3s/step - loss: 1.3572 - accuracy: 0.4028 - val_loss: 1.4434 - val_accuracy: 0.4167
Epoch 5/10
5/5 [==============================] - 17s 3s/step - loss: 1.0506 - accuracy: 0.6944 - val_loss: 1.4401 - val_accuracy: 0.4167
Epoch 6/10
5/5 [==============================] - 18s 3s/step - loss: 0.7032 - accuracy: 0.7986 - val_loss: 1.7172 - val_accuracy: 0.4722
Epoch 7/10
5/5 [==============================] - 16s 3s/step - loss: 0.4871 - accuracy: 0.8264 - val_loss: 1.9325 - val_accuracy: 0.4444
Epoch 8/10
5/5 [==================

In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [36]:
# Latih ulang model menggunakan data pelatihan yang telah diproses
model.fit(X_train_processed, y_train, epochs=10, validation_data=(X_valid_processed, y_valid))


Epoch 1/10
5/5 [==============================] - 18s 3s/step - loss: 6.0259e-04 - accuracy: 1.0000 - val_loss: 4.9898 - val_accuracy: 0.2778
Epoch 2/10
5/5 [==============================] - 17s 3s/step - loss: 2.8380e-04 - accuracy: 1.0000 - val_loss: 4.7548 - val_accuracy: 0.3333
Epoch 3/10
5/5 [==============================] - 17s 3s/step - loss: 1.5068e-04 - accuracy: 1.0000 - val_loss: 5.0715 - val_accuracy: 0.3333
Epoch 4/10
5/5 [==============================] - 17s 3s/step - loss: 1.3208e-04 - accuracy: 1.0000 - val_loss: 4.8203 - val_accuracy: 0.3889
Epoch 5/10
5/5 [==============================] - 17s 3s/step - loss: 1.1714e-04 - accuracy: 1.0000 - val_loss: 4.7135 - val_accuracy: 0.3889
Epoch 6/10
5/5 [==============================] - 17s 3s/step - loss: 1.0033e-04 - accuracy: 1.0000 - val_loss: 4.9802 - val_accuracy: 0.3889
Epoch 7/10
5/5 [==============================] - 19s 3s/step - loss: 7.9808e-05 - accuracy: 1.0000 - val_loss: 5.3023 - val_accuracy: 0.3889
Epoch 

In [37]:
# Evaluasi model
loss, accuracy = model.evaluate(X_valid_processed, y_valid)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


2/2 [==============================] - 1s 105ms/step - loss: 5.1973 - accuracy: 0.3889
Loss: 5.197301864624023
Accuracy: 0.3888888955116272


In [41]:
# Simpan model ke file
model.save('/content/race_identification_model.h5')


In [42]:
from google.colab import files

# Unduh file model ke perangkat lokal
files.download('race_identification_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>